In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import datetime
import gc

In [0]:
train_df = pd.read_csv("/content/drive/My Drive/Ashrae data/train.csv")

# Remove outliers
train_df = train_df [ train_df['building_id'] != 1099 ]
train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

building_df = pd.read_csv('/content/drive/My Drive/Ashrae data/building_metadata.csv')
weather_df = pd.read_csv('/content/drive/My Drive/Ashrae data/weather_train.csv')

In [0]:
weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
weather_df["day"] = weather_df["datetime"].dt.day
weather_df["week"] = weather_df["datetime"].dt.week
weather_df["month"] = weather_df["datetime"].dt.month
air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
#weather_df.update(air_temperature_filler,overwrite=False)

In [0]:
def fill_weather_dataset(weather_df):
  
  # Find Missing Dates
  time_format = "%Y-%m-%d %H:%M:%S"
  start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
  end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
  total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
  hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

  missing_hours = []
  for site_id in range(16):
      site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
      new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
      new_rows['site_id'] = site_id
      weather_df = pd.concat([weather_df,new_rows])

      weather_df = weather_df.reset_index(drop=True)           

  # Add new Features
  weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
  weather_df["day"] = weather_df["datetime"].dt.day
  weather_df["week"] = weather_df["datetime"].dt.week
  weather_df["month"] = weather_df["datetime"].dt.month

  # Reset Index for Fast Update
  weather_df = weather_df.set_index(['site_id','day','month'])

  air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
  weather_df.update(air_temperature_filler,overwrite=False)

  # Step 1
  cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
  # Step 2
  cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

  weather_df.update(cloud_coverage_filler,overwrite=False)

  due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
  weather_df.update(due_temperature_filler,overwrite=False)

  # Step 1
  sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
  # Step 2
  sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

  weather_df.update(sea_level_filler,overwrite=False)

  wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
  weather_df.update(wind_direction_filler,overwrite=False)

  wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
  weather_df.update(wind_speed_filler,overwrite=False)

  # Step 1
  precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
  # Step 2
  precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

  weather_df.update(precip_depth_filler,overwrite=False)

  weather_df = weather_df.reset_index()
  weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
      
  return weather_df

In [0]:
def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["weekend"] = df["timestamp"].dt.weekday
    holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                    "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                    "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                    "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                    "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                    "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                    "2019-01-01"]
    df["is_holiday"] = (df.timestamp.isin(holidays)).astype(int)
    df['square_feet'] =  np.log1p(df['square_feet'])
    
    # Remove Unused Columns
    drop = ["year_built","floor_count"]
    df = df.drop(drop, axis=1)
    gc.collect()
    
    # Encode Categorical Data
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df

In [0]:
weather_df = fill_weather_dataset(weather_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [0]:
weather_df

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,timestamp,wind_direction,wind_speed
0,0,25.000000,6.000000,20.000000,-0.173913,1019.700000,2016-01-01 00:00:00,0.000000,0.000000
1,0,24.400000,4.285714,21.100000,-1.000000,1020.200000,2016-01-01 01:00:00,70.000000,1.500000
2,0,22.800000,2.000000,21.100000,0.000000,1020.200000,2016-01-01 02:00:00,0.000000,0.000000
3,0,21.100000,2.000000,20.600000,0.000000,1020.100000,2016-01-01 03:00:00,0.000000,0.000000
4,0,20.000000,2.000000,20.000000,-1.000000,1020.000000,2016-01-01 04:00:00,250.000000,2.600000
...,...,...,...,...,...,...,...,...,...
140539,15,-3.534783,4.000000,-7.013043,-1.000000,1019.817391,2016-12-09 10:00:00,290.869565,6.565217
140540,15,-5.856522,1.500000,-8.982609,0.066667,1015.613043,2016-12-17 07:00:00,157.391304,5.695652
140541,15,-8.865217,0.000000,-12.730435,3.800000,1032.981818,2016-12-20 06:00:00,135.652174,3.500000
140542,15,2.630435,1.200000,-2.760870,-1.000000,1020.921739,2016-12-24 05:00:00,200.869565,4.847826


In [0]:
train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_df
gc.collect()

0

In [0]:
train_df_temp = features_engineering(train_df)

In [0]:
train_df_temp.head(10)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,weekend,is_holiday
0,105,0,2016-01-01,23.3036,1,0,10.832181,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
1,106,0,2016-01-01,0.3746,1,0,8.589514,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
2,106,3,2016-01-01,0.0000,1,0,8.589514,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
3,107,0,2016-01-01,175.1840,1,0,11.487946,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
4,108,0,2016-01-01,91.2653,1,0,11.309352,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
5,109,0,2016-01-01,80.9300,1,0,10.950736,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
6,109,3,2016-01-01,0.0000,1,0,10.950736,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
7,110,0,2016-01-01,86.2283,1,0,10.233331,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
8,111,0,2016-01-01,167.3920,1,0,11.681309,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
9,112,0,2016-01-01,10.2748,1,0,10.379939,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1


In [0]:
train_df = train_df_temp.drop(columns="timestamp")

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,weekend,is_holiday
0,105,0,2016-01-01,23.3036,1,0,10.832181,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
1,106,0,2016-01-01,0.3746,1,0,8.589514,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
2,106,3,2016-01-01,0.0000,1,0,8.589514,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
3,107,0,2016-01-01,175.1840,1,0,11.487946,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
4,108,0,2016-01-01,91.2653,1,0,11.309352,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1


In [0]:
train_df_temp.to_csv('/content/drive/My Drive/Ashrae data/train_tidy_temp.csv')

In [0]:
train_df.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,weekend,is_holiday
0,105,0,23.3036,1,0,10.832181,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
1,106,0,0.3746,1,0,8.589514,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
2,106,3,0.0000,1,0,8.589514,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
3,107,0,175.1840,1,0,11.487946,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1
4,108,0,91.2653,1,0,11.309352,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,0,4,1


In [0]:
target = np.log1p(train_df["meter_reading"])
features = train_df.drop('meter_reading', axis = 1)
del train_df
gc.collect()

11

In [0]:
categorical_features = ["building_id", "site_id", "meter", "primary_use", "is_holiday", "weekend"]
params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 1280,
    "learning_rate": 0.05,
    "feature_fraction": 0.85,
    "reg_lambda": 2,
    "metric": "rmse",
}

kf = KFold(n_splits=3)
models = []
for train_index,test_index in kf.split(features):
    train_features = features.loc[train_index]
    train_target = target.loc[train_index]
    
    test_features = features.loc[test_index]
    test_target = target.loc[test_index]
    
    d_training = lgb.Dataset(train_features, label=train_target,categorical_feature=categorical_features, free_raw_data=False)
    d_test = lgb.Dataset(test_features, label=test_target,categorical_feature=categorical_features, free_raw_data=False)
    
    model = lgb.train(params, train_set=d_training, num_boost_round=1000, valid_sets=[d_training,d_test], verbose_eval=25, early_stopping_rounds=50)
    models.append(model)
    del train_features, train_target, test_features, test_target, d_training, d_test
    gc.collect()

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[25]	training's rmse: 1.12212	valid_1's rmse: 1.27284
[50]	training's rmse: 0.887841	valid_1's rmse: 1.1305
[75]	training's rmse: 0.815674	valid_1's rmse: 1.11259
[100]	training's rmse: 0.769912	valid_1's rmse: 1.11139
[125]	training's rmse: 0.73784	valid_1's rmse: 1.11177
[150]	training's rmse: 0.716694	valid_1's rmse: 1.11416
Early stopping, best iteration is:
[102]	training's rmse: 0.766238	valid_1's rmse: 1.11112
Training until validation scores don't improve for 50 rounds.
[25]	training's rmse: 1.12376	valid_1's rmse: 1.2448
[50]	training's rmse: 0.889033	valid_1's rmse: 1.07888
[75]	training's rmse: 0.822059	valid_1's rmse: 1.05023
[100]	training's rmse: 0.782655	valid_1's rmse: 1.04205
[125]	training's rmse: 0.754225	valid_1's rmse: 1.03922
[150]	training's rmse: 0.735216	valid_1's rmse: 1.03822
[175]	training's rmse: 0.719319	valid_1's rmse: 1.03822
[200]	training's rmse: 0.708387	valid_1's rmse: 1.03829
Early stoppi

In [0]:
del features, target
gc.collect()

0